In [3]:
import sys
sys.path.append('../Classes')
from AzureSearch import Config
import pandas as pd
import openai
from azure.identity import DefaultAzureCredential

config = Config(path="../Classes/environment.env")
cred = DefaultAzureCredential(exclude_interactive_browser_credential=True)
token = cred.get_token("https://cognitiveservices.azure.com/.default")

openai.api_key = token.token
openai.api_version = config.openai_api_version
openai.api_type = "azure_ad"
openai.api_base = config.openai_api_base

def generate_grounded_answer(query, content, system, reminder, config=config):
    # Format the user message using the given query and content
    user_message = f"Source:\n{content}\n Answer the User question: {query}"

    # Construct the message to be sent to the model
    messages = [
        {"role": "system", "content": f"""{system}"""},
        {"role": "user", "content": f"""{user_message}\n{reminder}"""}
    ]

    # Request a response from the model
    response = openai.ChatCompletion.create(
        engine=config.gpt,
        messages=messages,
        temperature=0,
    )
    
    # Extract and return the model's reply
    bot_reply = response['choices'][0]['message']['content']
    return bot_reply

def process_queries_and_generate_responses(queries_titles, prompts, config=config):
    responses = []  # This list will store the final responses for each query and title
    queries_titles_df = pd.read_csv(queries_titles, encoding='utf-8')
    # Iterate over each row in the queries and titles DataFrame
    for _, row in queries_titles_df.iterrows():
        query = row['query']
        title = row['title']
        content = row['content']
        
        # For each prompt in the prompts DataFrame, generate a response using the mock OpenAI function
        prompt_responses = {}
        prompts_df = pd.read_csv(prompts, encoding='utf-8')
        for index, prompt_row in prompts_df.iterrows():
            prompt = prompt_row['prompt']
            reminder = prompt_row['reminder']
            ai_response = generate_grounded_answer(query, content, prompt, reminder, config)
            # Assuming you want the row num to start from 1
            column_name = f'prompt_{index + 1}'  # Adding 1 if the DataFrame index starts at 0
            prompt_responses[column_name] = ai_response

        # Compile the query, title, and all prompt responses into a single record
        record = {
            'query': query,
            'title': title,
            **prompt_responses  # Add the prompt responses with dynamic keys
        }
        responses.append(record)

    # Create a DataFrame from the responses
    final_df = pd.DataFrame(responses)

    return final_df

process_queries_and_generate_responses('./test_csv.txt', './test_csv_prompt.txt')

,query,title,prompt_1,prompt_2
0,what is the debt recovery loan code?,Debt_Lending,The debt recovery loan code is 20.,The debt recovery loan code is 20. Is there an...
